In [ ]:
pip install h2o


In [ ]:
pip install optuna

In [ ]:
import h2o
from h2o.estimators import H2OExtendedIsolationForestEstimator
import optuna
# Inicializar H2O
h2o.init()

In [ ]:
# Cargar datos
data = h2o.import_file('AllData/ProcesadoMatrizZ/DatosSanos75.csv')

#eliminar kurtosis
# Obtener el número de columnas en el H2OFrame
#num_columns = data.ncol

# Identificar las columnas que son múltiplos de 3
#indices_a_eliminar = [i for i in range(num_columns) if (i + 1) % 3 == 0]

# Convertir los índices a eliminar en un objeto H2OFrame
#cols_to_drop = [data.columns[i] for i in indices_a_eliminar]

# Eliminar las columnas identificadas
#data = data.drop(cols_to_drop)
data=data.drop(72,axis=1)#48-72

In [ ]:
data_validar =  h2o.import_file('AllData/ProcesadoMatrizZ/DatosSanos10Validar.csv')

#eliminar kurtosis
#num_columns = data_validar.ncol

# Identificar las columnas que son múltiplos de 3
#indices_a_eliminar = [i for i in range(num_columns) if (i + 1) % 3 == 0]

# Convertir los índices a eliminar en un objeto H2OFrame
#cols_to_drop_f = [data_validar.columns[i] for i in indices_a_eliminar]

# Eliminar las columnas identificadas
#data_validar = data_validar.drop(cols_to_drop_f)
data_validar=data_validar.drop(72,axis=1)#48-72


In [ ]:
# Define an objective function to be maximized.

def objective(trial):

    #1627680000000
    # # Inicializar H2O
    #h2o.init()
    # Cargar datos
    #data = h2o.import_file('.\Matrices\Matriz_training_PdP_norm.csv') trial.suggest_int('sample_size', 512, 1024)/// trial.suggest_int('extension_level', 40, 71)
    model = H2OExtendedIsolationForestEstimator(ntrees=trial.suggest_int('ntrees', 200, 500), seed=1627680000000000000,extension_level=trial.suggest_int('extension_level', 55, 65), sample_size=1024)
    model.train(training_frame=data)
    # Train Extended Isolation Forest
    predictions_training = model.predict(data)
    anomaly_score_training = predictions_training["anomaly_score"]
    corte1 = anomaly_score_training.max()
    #data_fallos =  h2o.import_file('.\Matrices\Matriz_val_PdP_norm.csv')
    predictions_val = model.predict(data_validar)
    anomaly_score_val = predictions_val["anomaly_score"]
    corte2 = anomaly_score_val.max()
    accuracy = max(corte1, corte2)
    return accuracy

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

In [ ]:
from optuna.trial import TrialState
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

In [ ]:
model

In [ ]:
data_testeo =  h2o.import_file('AllData/ProcesadoMatrizZ/DatosFallo3.csv')

#eliminar kurtosis
num_columns = data_testeo.ncol

# Identificar las columnas que son múltiplos de 3
indices_a_eliminar = [i for i in range(num_columns) if (i + 1) % 3 == 0]

# Convertir los índices a eliminar en un objeto H2OFrame
cols_to_drop_f = [data_testeo.columns[i] for i in indices_a_eliminar]

# Eliminar las columnas identificadas
data_testeo = data_testeo.drop(cols_to_drop_f)
data_testeo=data_testeo.drop(48,axis=1)
# Realizar predicciones de testeo
predictions_testeo = model.predict(data_testeo)
predictions_testeo

In [ ]:
# Contar cuántos valores de anomaly_score superan 0.65
count_above_testeo = (predictions_testeo['anomaly_score'] > 0.57).sum()
count_above_testeo

In [ ]:
#NO USAR
# Crear modelo
#1627680000000000000
#1.62768e+18
model = H2OExtendedIsolationForestEstimator(ntrees=100, seed=1627680000000000000 ,extension_level=47, sample_size=1024)

# Entrenar modelo
model.train(training_frame=data)
# Train Extended Isolation Forest
#model.train(x = data.columns, training_frame = data)


# Realizar predicciones
predictions = model.predict(data)

anomaly_score = predictions["anomaly_score"]
anomaly_score.max()


# Contar cuántos valores de anomaly_score superan 0.6
count_above_threshold = (predictions['anomaly_score'] > 0.65).sum()
count_above_threshold
